### **Nombre:** Andrés Felipe Riaño Quintanilla.
### **Cédula:** 1083928808.

# Laboratorio 07

**Librerías:**

In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pylab as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.datasets import load_breast_cancer

**Clase:**

In [2]:
class Neural_Network():
    '''
    Clase diseñada para realizar un proceso de clasificación binaria utilizando una red neuronal, 
    a partir de datos de entrenamiento y prueba proporcionados.

    Parámetros:
    ----------
    N_epoch : int
        Número de épocas para entrenar el algoritmo.
    structure : numpy.ndarray
        Arreglo con el número de neuronas que tendrá cada capa.
    m_train : int
        Número de datos de entrenamiento.
    m_test : int
        Número de datos de prueba.
    Xtrain : numpy.ndarray
        Arreglo con los datos de entrenamiento. Las filas deben ser las características y las
        columnas los datos.
    ytrain : numpy.ndarray
        Arreglo con las etiquetas de los datos de entrenamiento.
    Xtest : numpy.ndarray
        Arreglo con los datos de prueba. Las filas deben ser las características y las 
        columnas los datos.
    ytest : numpy.ndarray
        Arreglo con las etiquetas de los datos de prueba.
    alpha : float
        Tasa de aprendizaje del gradiente descendente.

    Todos los parámetros se convierten en atributos del objeto.

    Otros atributos:
    ----------------
    w : numpy.ndarray of object
        Almacena las matrices de pesos asociadas a cada capa después de haber ejecutado el método
        Binary_Classification.

    b : numpy.ndarray of object
        Almacena los vectores de sesgos asociados a cada capa después de haber ejecutado el método
        Binary_Classification.
    '''

    def __init__(self,N_epoch,structure,Xtrain,ytrain,Xtest,ytest,alpha):
        '''
        Inicializa la clase con los parámetros dados. 
        '''
        self.N_epoch = N_epoch
        self.structure = structure
        self.m_train = len(ytrain)
        self.m_test = len(ytest)
        self.Xtrain = Xtrain
        self.ytrain = ytrain
        self.Xtest = Xtest
        self.ytest = ytest
        self.alpha = alpha

    def ReLU(self, x):
        '''
        Calcula el valor de la función de activación rectificadora evaluada en la entrada x.

        Parámetros:
        -----------
        x : float
            Valor real a evaluar.
        
        Retorna:
        -------- 
        float
            El valor de entrada si es mayor o igual a 0, o 0 si es negativo.
        '''
        if x >= 0:
        
            return x
        
        else:
        
            return 0
    
    def Sigmoid(self, x):
        '''
        Calcula el valor de la función sigmoide evaluada en la entrada x.

        Parámetros:
        -----------
        x : float
            Valor real a evaluar.

        Retorna :
        --------
        sigma : float
            Valor de la función sigmoide evaluada en x.
        '''
        sigma = 1/(1 + np.exp(-x))
        
        return sigma
    
    def DReLU(self, x):
        '''
        Calcula el valor de la derivada de la función de activación rectificadora evaluada
        en la entrada x.

        Parámetros:
        -----------
        x : float
            Valor real a evaluar.

        Retorna:
        --------
        float
            1 si el valor de entrada es mayor o igual que cero, o 0 en caso contrario.
        '''
        if x >= 0:

            return 1
        
        else:

            return 0
        
    def Cost(self, A_L):
        '''
        Calcula el valor de la función de coste para los datos de entrenamiento ingresados en
        una época determinada.

        Parámetros:
        -----------
        A_L : numpy.ndarray
            Vector fila con los valores de salida de la red neuronal.

        Retorna:
        --------
        J : float
            Valor de la función de coste evaluada en los datos de entrenamiento y en la salida
            de la red neuronal en una época determinada.
        '''
        J = - (1/self.m_train)*np.sum(self.ytrain*np.log(A_L) + (1 - self.ytrain)*np.log(1 - A_L))

        return J
        
    def Forward(self, g, A_l_1, W_l, b_l):
        '''
        Dada una función de activación g, la salida A_l_1 de la capa l-1, la matriz de 
        pesos de la capa l W_l y el vector de sesgos de la capa l b_l, devuelve Z_l y A_l,
        que son la entrada de la función de activación y la salida de la capa l,
        respectivamente.

        Parámetros:
        -----------
        g : callable
            Función de activación sin vectorizar.
        A_l_1 : numpy.ndarray
            Matriz de salida de la capa l-1.
        W_l : numpy.ndarray
            Matriz de pesos de la capa l.
        b_l : numpy.ndarray
            Vector de sesgos de la capa l.
        
        Retorna:
        --------
        A_l : numpy.ndarray
            Matriz de salida de la capa l.
        Z_l : numpy.ndarray
            Entrada de la función de activación de la capa l.
        '''
        Z_l = W_l@A_l_1 + b_l
        A_l = np.vectorize(g)(Z_l)

        return A_l, Z_l 
    
    def Backward_L(self, delta_L, A_L_1, W_L, b_L):
        '''
        Efectúa el proceso de retropropagación de la capa de salida de la red neuronal.

        Parámetros:
        -----------
        delta_L : numpy.ndarray
            Diferencia entre la salida de la red A_L y el vector fila de datos etiquetados y.
        A_L_1 : numpy.ndarray
            Matriz de salida de la penúltima capa A_L_1.
        W_L : numpy.ndarray
            Matriz de pesos de la última capa L.
        b_L : numpy.ndarray
            Vector de sesgos de la última capa L.

        Retorna: 
        --------
        W_L : numpy.ndarray
            Matriz de pesos de la capa L con la corrección del gradiente descendente.
        b_L : numpy.ndarray
            Vector de sesgos de la capa L con la corrección del gradiente descendente.
        '''
        dJdW = ((1/self.m_train)*delta_L) @ (A_L_1.T)
        dJdb = (1/self.m_train)*np.sum(delta_L, axis=0)
        W_L = W_L - (self.alpha * dJdW)
        b_L = b_L - (self.alpha  * dJdb)

        return W_L, b_L
    
    def Backward(self, delta_l1, W_l1, Z_l, A_l_1, W_l, b_l):
        '''
        Efectúa el proceso de retropropagación en las capas ocultas de la red neuronal.

        Parámetros: 
        -----------
        delta_l1 : numpy.ndarray
            Denota el producto de Hadamard entre el producto de la transpuesta de la 
            matriz de pesos en la capa l+2 con el mismo vector delta, pero de la capa
            l+2 y la derivada de la función de activación de la capa l+1 evaluada 
            en la matriz de entrada Z de la función de activación de la capa l+1
            (\delta^{[l+1]}=( W^{[l+2]})^T \delta^{[l+2]} \odot g'( Z^{[l+1]})).
        W_l1 : numpy.ndarray
            Matriz de pesos de la capa l+1.
        Z_l : numpy.ndarray
            Entrada de la función de activación de la capa l. 
        A_l_1 : numpy.ndarray
            Matriz de salida de la capa l-1.
        W_l : numpy.ndarray
            Matriz de pesos de la capa l.
        b_l : numpy.ndarray
            Vector de sesgos de la capa l.

        Retorna:
        --------
        W_l : numpy.ndarray
            Matriz de pesos de la capa l con la corrección del gradiente descendente.
        b_l : numpy.ndarray
            Vector de sesgos de la capa l con la corrección delg radiente descendente.
        delta_l : numpy.ndarray
            Denota el producto de Hadamard entre el producto de la transpuesta de la 
            matriz de pesos en la capa l+1 con el mismo vector delta, pero de la capa
            l+1 y la derivada de la función de activación de la capa l evaluada 
            en la matriz de entrada Z de la función de activación de la capa l
            (\delta^{[l]}=( W^{[l+1]})^T \delta^{[l+1]} \odot g'( Z^{[l]})).
        '''
        delta_l = (W_l1.T @ delta_l1) * np.vectorize(self.DReLU)(Z_l)
        dJdW = ((1/self.m_train)*delta_l) @ (A_l_1.T)
        dJdb = (1/self.m_train)*np.sum(delta_l, axis=0)
        W_l = W_l - (self.alpha * dJdW)
        b_l = b_l - (self.alpha  * dJdb)

        return W_l, b_l, delta_l
    
    def Initialization(self):
        '''
        Inicializa la red neuronal con pesos y sesgos aleatorios y realiza el proceso de 
        retropropagación una vez.

        Retorna:
        --------
        W_n : numpy.ndarray of object
            Objeto que almacena en cada una de sus entradas a las diferentes matrices de 
            pesos de cada capa.
        b_n : numpy.ndarray of object
            Objeto que almacena en cada una de sus entradas a los diferentes vectores de
            sesgos de cada capa.
        '''
        A_0 = self.Xtrain
        A = np.empty(len(self.structure) ,dtype = object)
        Z = np.empty(len(self.structure) ,dtype = object)
        W = np.empty(len(self.structure) ,dtype = object)
        b = np.empty(len(self.structure) ,dtype = object)
        delta = np.empty(len(self.structure) ,dtype = object)

        A[0] = A_0
        Z[0] = 0
        W[0] = 0
        b[0] = 0

        for l,nl in enumerate(self.structure[1:-1], start = 1):

            nl_1 = np.shape(A[l-1])[0]
            W[l] = 1-2*np.random.random((nl,nl_1))
            b[l] = np.tile( 1-2*np.random.random((nl)), (self.m_train, 1)).T
            A[l],Z[l] = self.Forward(self.ReLU,A[l-1],W[l],b[l])

        nL = self.structure[-1]
        nL_1 = np.shape(A[-2])[0]
        W[-1] = 1-2*np.random.random((nL,nL_1))
        b[-1] = np.tile( 1-2*np.random.random((nL)), (self.m_train, 1)).T
        A[-1],Z[-1] = self.Forward(self.Sigmoid,A[-2],W[-1],b[-1])

        delta[-1] = A[-1] - self.ytrain
        W_n = np.empty(len(self.structure) ,dtype = object)
        b_n = np.empty(len(self.structure) ,dtype = object)
        W_n[0] = 0 
        b_n[0] = 0
        W_n[-1], b_n[-1] = self.Backward_L(delta[-1],A[-2],W[-1],b[-1])

        for l in range(len(self.structure)-2,0,-1):

            W_n[l], b_n[l], delta[l] = self.Backward(delta[l+1],W[l+1],Z[l],A[l-1],W[l],b[l])

        return W_n, b_n 
    
    def bin(self, x):
        '''
        Dado un valor de entrada x, devuelve 0 si x es menor a 0.5 y 1 si es mayor a 0.5.

        Parámetros:
        x : float
            Valor real a evaluar.

        Retorna:
        --------
        float
            0 si el valor de entrada es menor que 0.5, o 1 en caso contrario.
        '''
        return 0 if x < 0.5 else 1
    
    def Binary_Classification(self):
        '''
        Imprime el puntaje de entrenamiento, el puntaje de prueba y el valor de la función
        de coste en cada época. Además almacena como atributos las matrices de pesos y los vectores 
        de sesgos de la última época, esto con el fin de guardar la estructura final de la red neuronal
        y poder hacer predicciones.
        '''
        A_0 = self.Xtrain
        A_0T = self.Xtest
        W, b = self.Initialization()

        for epoch in range(1,self.N_epoch+1):

            A = np.empty(len(self.structure) ,dtype = object)
            Z = np.empty(len(self.structure) ,dtype = object)

            A_test = np.empty(len(self.structure) ,dtype = object)
            Z_test = np.empty(len(self.structure) ,dtype = object)

            A[0] = A_0
            Z[0] = 0

            A_test[0] = A_0T
            Z_test[0] = 0

            for l in range(1,len(self.structure)-1):

                A[l],Z[l] = self.Forward(self.ReLU,A[l-1],W[l],b[l])
                A_test[l],Z_test[l] = self.Forward(self.ReLU,A_test[l-1],W[l],b[l][:,0:self.m_test])

            A[-1], Z[-1] = self.Forward(self.Sigmoid,A[-2],W[-1],b[-1])
            A_test[-1],Z_test[-1] = self.Forward(self.Sigmoid,A_test[-2],W[-1],b[-1][:,0:self.m_test])
            Output_train = np.vectorize(self.bin)(A[-1])
            Output_test = np.vectorize(self.bin)(A_test[-1])
            
            Train_score = np.sum(Output_train == self.ytrain)/len(self.ytrain)
            Test_score = np.sum(Output_test == self.ytest)/len(self.ytest)
            cost_function = self.Cost(A[-1])

            print('Epoch: {}/{}. Train Score: {}. Test Score: {}. Cost function: {}'.format(epoch,self.N_epoch,Train_score,Test_score,cost_function))

            delta = np.empty(len(self.structure) ,dtype = object)
            delta[-1] = A[-1] - self.ytrain
            W_n = np.empty(len(self.structure) ,dtype = object)
            b_n = np.empty(len(self.structure) ,dtype = object)
            W_n[0] = 0 
            b_n[0] = 0
            W_n[-1], b_n[-1] = self.Backward_L(delta[-1],A[-2],W[-1],b[-1])
                
            for l in range(len(self.structure)-2,0,-1):

                W_n[l], b_n[l], delta[l] = self.Backward(delta[l+1],W[l+1],Z[l],A[l-1],W[l],b[l])

            W = W_n
            b = b_n

        self.W = W
        self.b = b

    def Prediction(self, x):
        '''
        Una vez definida la estructura de la red neuronal al ejecutar el método 
        Binary_Classification, predice las etiquetas de los datos ingresados x.

        Parámetros:
        -----------
        x : numpy.ndarray
            Conjunto de datos a clasificar.
        
        Retorna:
        --------
        numpy.ndarray
            Vector cuyas entradas corresponden a la clasificación de cada uno de los datos ingresados.
        '''
        mx = np.shape(x)[1]
        A = np.empty(len(self.structure) ,dtype = object)
        Z = np.empty(len(self.structure) ,dtype = object)
        A[0] = x

        for l in range(1,len(self.structure)-1):

            A[l], Z[l] = self.Forward(self.ReLU,A[l-1],self.W[l],self.b[l][:,0:mx])

        A[-1], Z[-1] = self.Forward(self.Sigmoid,A[-2],self.W[-1],self.b[-1][:,0:mx])

        return np.vectorize(self.bin)(A[-1])

**Ejemplos de ejecución:**

**1.** Dataset de fotos de gatos:

In [3]:
data_train= "train_catvnoncat.h5"
train_dataset = h5py.File(data_train, "r")

data_test= "test_catvnoncat.h5"
test_dataset = h5py.File(data_test, "r")

xtrain_classes, xtrain, train_label =\
train_dataset["list_classes"],train_dataset["train_set_x"],train_dataset["train_set_y"]

test_classes, xtest,test_label =\
test_dataset["list_classes"],test_dataset["test_set_x"],test_dataset["test_set_y"]

ytrain = np.array(list(train_label))
Xtrain = (np.reshape(xtrain,(209, 64*64*3))/255).T

ytest = np.array(list(test_label))
Xtest = (np.reshape(xtest,(50, 64*64*3))/255).T

structure = np.array([12288,10,10,1])   #4 capas: 1 de entrada, 1 de salida y dos capas ocultas. Las dos capas ocultas tienen 10 neuronas.

In [4]:
modelo = Neural_Network(100,structure,Xtrain,ytrain,Xtest,ytest,10)

In [5]:
modelo.Binary_Classification()

/tmp/ipykernel_7107/1768599265.py:135: RuntimeWarning: divide by zero encountered in log
  J = - (1/self.m_train)*np.sum(self.ytrain*np.log(A_L) + (1 - self.ytrain)*np.log(1 - A_L))
/tmp/ipykernel_7107/1768599265.py:135: RuntimeWarning: invalid value encountered in multiply
  J = - (1/self.m_train)*np.sum(self.ytrain*np.log(A_L) + (1 - self.ytrain)*np.log(1 - A_L))


Epoch: 1/100. Train Score: 0.3444976076555024. Test Score: 0.66. Cost function: nan
Epoch: 2/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.631371267862544
Epoch: 3/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.6209227314606658
Epoch: 4/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.6107146849831707
Epoch: 5/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.6007416311841925
Epoch: 6/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.5909981374554951
Epoch: 7/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.5814788400321538
Epoch: 8/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.5721784478331683
Epoch: 9/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.5630917459424555
Epoch: 10/100. Train Score: 0.6555023923444976. Test Score: 0.34. Cost function: 0.5542135987373948
Epoch: 11/100. Train Scor

El puntaje de entrenamiento es 1, pero el puntaje de prueba es 0.42. Esto evidencia un sobreajuste en el modelo. 

In [6]:
print('Predicción: {}\nRealidad: {}'.format(modelo.Prediction(Xtest[:,0:10])[0],ytest[0:10]))

Predicción: [0 0 1 0 0 0 0 1 0 0]
Realidad: [1 1 1 1 1 0 1 1 1 1]


Se puede ver que el algoritmo no es muy eficiente.

**2.** Dataset de cáncer de mama.

In [7]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [8]:
split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df, df["target"]):
  strat_train_set = df.loc[train_index]
  strat_test_set = df.loc[test_index]

df_train = strat_train_set   #Datos de entrenamiento.
df_test = strat_test_set   #Datos de prueba.

In [9]:
Xtrain2 = df_train.drop('target', axis=1).to_numpy().T
ytrain2 = df_train['target'].to_numpy()

Xtest2 = df_test.drop('target', axis=1).to_numpy().T
ytest2 = df_test['target'].to_numpy()

In [10]:
structure2 = np.array([30,2,2,1])

modelo2 = Neural_Network(100, structure2,Xtrain2,ytrain2,Xtest2,ytest2,10000)

In [11]:
modelo2.Binary_Classification()

Epoch: 1/100. Train Score: 0.6263736263736264. Test Score: 0.631578947368421. Cost function: nan
Epoch: 2/100. Train Score: 0.4153846153846154. Test Score: 0.37719298245614036. Cost function: inf
Epoch: 3/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 4/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 5/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 6/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 7/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 8/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 9/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 10/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 11/100. Train Score: 1.0. Test Score: 0.47368421052631576. Cost function: nan
Epoch: 12/100. Train Score: 1.0. Test Score: 0

/tmp/ipykernel_7107/1768599265.py:135: RuntimeWarning: divide by zero encountered in log
  J = - (1/self.m_train)*np.sum(self.ytrain*np.log(A_L) + (1 - self.ytrain)*np.log(1 - A_L))
/tmp/ipykernel_7107/1768599265.py:135: RuntimeWarning: invalid value encountered in multiply
  J = - (1/self.m_train)*np.sum(self.ytrain*np.log(A_L) + (1 - self.ytrain)*np.log(1 - A_L))
/tmp/ipykernel_7107/1768599265.py:92: RuntimeWarning: overflow encountered in exp
  sigma = 1/(1 + np.exp(-x))


Similar al caso anterior, se tiene un puntaje de entrenamiento muy alto y un puntaje de prueba muy bajo, lo que indica que este modelo también se sobreajusta.

In [12]:
print('Predicción: {}\nRealidad: {}'.format(modelo2.Prediction(Xtest2[:,0:10])[0],ytest2[0:10]))

Predicción: [1 0 1 1 1 1 1 1 1 1]
Realidad: [0 1 0 1 0 1 1 0 0 0]


Nuevamente, a partir de un ejemplo práctico se ve claramente que el modelo no es muy bueno.